In [117]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from collections import Counter

In [4]:
import statsmodels.api as sm
# from statsmodels.discrete.discrete_model import Logit
# from statsmodels.discrete.discrete_model import LogitResults
from sklearn.model_selection import train_test_split # X, y, random_state=9
from sklearn.preprocessing import StandardScaler # .fit(X_train), .transform(X) & y

from sklearn.metrics import accuracy_score # (y_test, y_pred)
from sklearn.metrics import classification_report # (y_test, y_pred)
from sklearn.metrics import confusion_matrix # (y_test, y_pred)

In [2]:
dfmodel = pd.read_pickle('../../data/ecommerce/dfmodel_script.pkl', compression='zip')

In [12]:
dfmodel.head(3)
# dfmodel.shape

,timestamp,product_count,transactionid,addtocart,transaction,view,time_delta,time_hour,made_purchase
visitorid,,,,,,,,,
0,2015-09-11 14:55:17,3,NaN,0,0,3,00:05:28,0.091111,0
2,2015-08-07 12:20:57,4,NaN,0,0,8,00:29:13,0.486944,0
6,2015-08-30 21:21:25,3,NaN,1,0,5,21:17:37,21.293611,0


In [11]:
df20 = dfmodel.sample(20000, random_state=46)
df20.head()
df20[df20['transaction'] > 0].shape

(623, 9)

In [41]:
col_all = ['product_count', 'addtocart', 'view', 'time_hour']

In [36]:
def sort_split(df):
    """ return holdout, crossval, tank
    data randomly split among groups for modelling
    2x upsample 11449 purchase rows
    380127 rows main dfmodel
    """
    intforrand = 9
    purmask = df['made_purchase'] == 1
    pur = df[purmask]
    nopur = df[~purmask]
    pur = pur.sample(frac=2., replace=True, random_state=intforrand) # upsample to 22898
    upsamp_rows_size = pur.shape[0]
    # split pur
    holdpur = pur.sample(frac=0.25, random_state=intforrand) #df
    holdmask = pur.index.isin(holdpur.index)
    crosspur = pur[~holdmask] #df
    # split nopur
    times4_size = upsamp_rows_size * 4
    nopur_formodel = nopur.sample(n=times4_size, random_state=intforrand) #  91592
    holdnopur = nopur_formodel.sample(frac=0.25, random_state=intforrand) #df
    holdmaskno = nopur_formodel.index.isin(holdpur.index)
    crossnopur = nopur_formodel[~holdmaskno]  #df
    # set aside tank data unused for model
    tankmask = nopur.index.isin(nopur_formodel.index)
    dftank = nopur[~tankmask]  #df
    # combine for crossval and holdout
    holdappended = holdnopur.append(holdpur)
    crossappended = crossnopur.append(crosspur)
    holdout = holdappended.sample(frac=1., random_state=intforrand)
    crossval = crossappended.sample(frac=1., random_state=intforrand)
    return holdout, crossval, dftank

In [22]:
purmask = dfmodel['made_purchase'] == 1
pur = dfmodel[purmask]
print(dfmodel[~purmask].shape)
print(pur.shape)
print(dfmodel.shape)

(368678, 9)
(11449, 9)
(380127, 9)


In [35]:
testq = pur.sample(frac=0.25, random_state=9)
pur.sample(frac=1., replace=False, random_state=9).shape

(11449, 9)

In [31]:
mask = pur.index.isin(testq.index)
mask.shape

(11449,)

In [33]:
pur[~mask].shape

(8587, 9)

In [38]:
holdout20, crossval20, tank20 = sort_split(df20)
print(holdout20.shape, crossval20.shape, tank20.shape)

(1558, 9) (5540, 9) (14393, 9)


In [60]:
# sm.Logit()
# sm.add_constant()
X20 = crossval20[col_all].to_numpy()
y20 = crossval20['made_purchase'].to_numpy().reshape(-1,1)
print(X20.shape, y20.shape)

(5540, 4) (5540, 1)


In [59]:
# crossval20[col_all].head().to_numpy()
crossval20[col_all].head(2)
# crossval20['made_purchase'].head()

,product_count,addtocart,view,time_hour
visitorid,,,,
1340751,1,0,2,120.808889
1062248,4,0,4,0.080278


In [61]:
# STANDARDIZE
scaler = StandardScaler().fit(X20)
X20stan = scaler.transform(X20)

In [52]:
X20 = sm.add_constant(X20)
logit20 = sm.Logit(y20, X20) # endog, exog

Logit.fit(start_params=None, method='newton', maxiter=35, full_output=1, disp=1, callback=None, **kwargs)  
https://www.statsmodels.org/stable/generated/statsmodels.discrete.discrete_model.Logit.fit.html#statsmodels.discrete.discrete_model.Logit.fit

In [54]:
logit20_res = logit20.fit()

Optimization terminated successfully.
         Current function value: inf
         Iterations 7


/home/scorpio/enter/lib/python3.7/site-packages/statsmodels/discrete/discrete_model.py:1789: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


In [96]:
print('Parameters: ', logit20_res.params)
print(logit20_res.summary2())

Parameters:  [-2.99400471e+00 -9.24578176e-02  1.88308965e+00  6.85438195e-02
 -2.18715904e-04]
                       Results: Logit
Model:              Logit            Pseudo R-squared: inf   
Dependent Variable: y                AIC:              inf   
Date:               2020-02-06 20:22 BIC:              inf   
No. Observations:   5540             Log-Likelihood:   -inf  
Df Model:           4                LL-Null:          0.0000
Df Residuals:       5535             LLR p-value:      1.0000
Converged:          1.0000           Scale:            1.0000
No. Iterations:     7.0000                                   
--------------------------------------------------------------
          Coef.   Std.Err.     z      P>|z|    [0.025   0.975]
--------------------------------------------------------------
const    -2.9940    0.0699  -42.8284  0.0000  -3.1310  -2.8570
x1       -0.0925    0.0243   -3.7976  0.0001  -0.1402  -0.0447
x2        1.8831    0.0826   22.8020  0.0000   1.7212  

In [58]:
logit20.loglikeobs(logit20_res.params)[:50]

array([           -inf,            -inf,            -inf,            -inf,
        0.00000000e+00, -1.71286791e+00,            -inf,            -inf,
                  -inf,            -inf, -1.37007653e+00,            -inf,
                  -inf,            -inf, -6.42403836e+02,            -inf,
       -3.44723624e+02,            -inf, -7.06787814e+02, -1.66581986e+00,
                  -inf,            -inf,            -inf, -3.51225582e-01,
       -1.41359130e+00,            -inf,            -inf,            -inf,
       -3.77724571e-04,            -inf,            -inf,            -inf,
                  -inf,            -inf,            -inf,            -inf,
                  -inf,            -inf,            -inf,  0.00000000e+00,
                  -inf,            -inf,            -inf, -3.88550471e-01,
                  -inf,            -inf,            -inf, -2.89382664e+02,
                  -inf,            -inf])

In [62]:
X20stan = sm.add_constant(X20stan)
logit20stan = sm.Logit(y20, X20stan) # endog, exog
logit20stan_res = logit20stan.fit()

Optimization terminated successfully.
         Current function value: inf
         Iterations 7


/home/scorpio/enter/lib/python3.7/site-packages/statsmodels/discrete/discrete_model.py:1789: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


In [95]:
print('Parameters: ', logit20stan_res.params)
print(logit20stan_res.summary2())

Parameters:  [-2.53437741 -0.72358836  2.09519975  0.73614315 -0.11129846]
                       Results: Logit
Model:              Logit            Pseudo R-squared: inf   
Dependent Variable: y                AIC:              inf   
Date:               2020-02-06 20:22 BIC:              inf   
No. Observations:   5540             Log-Likelihood:   -inf  
Df Model:           4                LL-Null:          0.0000
Df Residuals:       5535             LLR p-value:      1.0000
Converged:          1.0000           Scale:            1.0000
No. Iterations:     7.0000                                   
--------------------------------------------------------------
          Coef.   Std.Err.     z      P>|z|    [0.025   0.975]
--------------------------------------------------------------
const    -2.5344    0.0572  -44.3265  0.0000  -2.6464  -2.4223
x1       -0.7236    0.1905   -3.7976  0.0001  -1.0970  -0.3501
x2        2.0952    0.0919   22.8020  0.0000   1.9151   2.2753
x3        0.7

In [71]:
# logit20stan.loglikeobs(logit20stan_res.params)[:50]
# exact same as unscaled

In [80]:
print(np.exp(-0.11129846))
np.exp(logit20stan_res.params) # so addtocart & view makes the prediction

0.8946716853659036


array([0.07931108, 0.48500874, 8.12706421, 2.08786738, 0.89467169])

In [85]:
X20stan[:20, 4]

array([-0.1736138 , -0.41086118, -0.41095725, -0.32346313, -0.41092668,
        2.93823953, -0.41047416, -0.41098346,  0.27395705, -0.41086937,
       -0.31538153, -0.41093324,  2.37888283, -0.41088847, -0.35208251,
        0.26348837,  2.74192471, -0.41101621, -0.40328015,  0.38608135])

In [102]:
X20stan_prob = logit20stan_res.predict(X20stan)
X20stan_prob[:10]

array([0.04853131, 0.04353503, 0.04557109, 0.04711298, 0.9185278 ,
       0.18034783, 0.04353324, 0.04557122, 0.03955094, 0.04557067])

In [84]:
X20stan[4:6]

array([[ 1.        ,  0.12535505,  2.46381328, -0.21778814, -0.41092668],
       [ 1.        ,  0.50868551,  0.66628571,  0.43399641,  2.93823953]])

In [124]:
print(X20stan_prob.shape)
X20stan_pred = np.uint8(X20stan_prob.reshape(-1,1) > 0.5)
# X20stan_prob.reshape(-1,1) == 1

(5540,)


In [127]:
accuracy_score(y20, X20stan_pred) # y_test, y_pred

0.9095667870036102

In [126]:
print(y20.shape, X20stan_pred.shape)
print(Counter(y20[:, 0]), Counter(X20stan_pred[:, 0]))

(5540, 1) (5540, 1)
Counter({0: 4984, 1: 556}) Counter({0: 5325, 1: 215})


In [129]:
print(classification_report(y20, X20stan_pred))

              precision    recall  f1-score   support

           0       0.92      0.98      0.95      4984
           1       0.63      0.24      0.35       556

    accuracy                           0.91      5540
   macro avg       0.77      0.61      0.65      5540
weighted avg       0.89      0.91      0.89      5540



In [130]:
print(confusion_matrix(y20, X20stan_pred))

[[4904   80]
 [ 421  135]]
